## Abstract

We download action movie data from **IMDb** and **Rotten Tomatoes**. We select 8 attrbutes here:

- URL: where we download this tuple of data
- Name: movie name
- Year: when this movie comes out
- Time: how long the movie last
- Rating
- Genre
- Director
- Stars

In [1]:
## Packages we used here
import requests
import re
import json
import csv

In [2]:
## Extract information from a list of searching url -- IMDb
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3'}
imdb_url = ['https://www.imdb.com/search/title?title_type=feature,tv_movie&genres=action&languages=en&start=' + str(i) + '&ref_=adv_nxt' for i in range(1, 3100, 50)]
imdb_result = list()
for url in imdb_url:
    text = requests.get(url, headers = headers).text
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = text.replace('\t', '')
    header = re.findall('<h3 class="lister-item-header">(((?!</h3>).)*)</h3>', text)
    len_type = re.findall('<p class="text-muted ">(((?!</p>).)*)</p>', text)
    director_star = re.findall('<p class="">(((?!</p>).)*)</p>', text)
    for i in range(len(header)):
        item = dict()
        item['URL'] = url
        name = re.search('<a href="[^"]*">([^<>]*)</a>', header[i][0])
        if name is not None:
            item['Name'] = name.group(1).strip()
        year = re.search('<span class="lister-item-year text-muted unbold">([^<>]*)</span>', header[i][0])
        if year is not None:
            item['Year'] = re.search('[^0-9]*([0-9]*)[^0-9]*', year.group(1).strip()).group(1)
        time = re.search('<span class="runtime">([^<>]*)</span>', len_type[i][0])
        if time is not None:
            item['Time'] = time.group(1).strip()
        rating = re.search('<span class="certificate">([^<>]*)</span>', len_type[i][0])
        if rating is not None:
            item['Rating'] = rating.group(1).strip()
        genre = re.search('<span class="genre">([^<>]*)</span>', len_type[i][0])
        if genre is not None:
            item['Genre'] = genre.group(1).strip()
        split_ds = director_star[i][0].split('Stars')
        if director_star[i][0].find('Director') != -1:
            item['Director'] = re.search('<a href="[^"]*">([^<>]*)</a>', split_ds[0]).group(1).strip()
        if director_star[i][0].find('Stars') != -1:
            item['Stars'] = ','.join(re.findall('<a href="[^"]*">([^<>]*)</a>', split_ds[1])).strip()
        imdb_result.append(item)
        

In [3]:
## Save IMDb data into csv
with open('../data/tableA_IMDb.csv', 'w') as f:
    keys = imdb_result[0].keys()
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(imdb_result)

In [4]:
## Function of extract url of every movie from a list of searching url -- Rotten Tomatoes
tomato_url = ['http://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&maxPopcorn=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&genres=1&certified&sortBy=release&type=dvd-streaming-all&page=' + str(i) for i in range(2, 105)]
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
          'accept-encoding': 'gzip, deflate, br',
          'accept-language': 'zh-CN,zh;q=0.9',
          'cookie': 'akamai_generated_location={"zip":"53701-53708+53711+53713-53719+53725-53726+53744+53774+53777-53779+53782-53786+53788-53794","city":"MADISON","state":"WI","county":"DANE","areacode":"608","lat":"43.0733","long":"-89.4012","countrycode":"US"}; akacd_RTReplatform=2177452799~rv=87~id=4edea432f05c395924a44df78ba30c60; optimizelyEndUserId=oeu1555115688922r0.8815088990364477; optimizelySegments=%7B%22577540156%22%3A%22none%22%2C%22589340523%22%3A%22false%22%2C%22589910512%22%3A%22direct%22%2C%22595650510%22%3A%22gc%22%2C%22697031154%22%3A%22true%22%7D; optimizelyBuckets=%7B%7D; _cb_ls=1; AMCVS_8CF467C25245AE3F0A490D4C%40AdobeOrg=1; _fbp=fb.1.1555115689245.635350704; adops_master_kvs=; AMCV_8CF467C25245AE3F0A490D4C%40AdobeOrg=-1303530583%7CMCIDTS%7C18000%7CMCMID%7C20483888317875100373723114311517804425%7CMCAAMLH-1555720489%7C7%7CMCAAMB-1555720489%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1555122889s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C3.3.0; s_vnum=1557707689567%26vn%3D1; s_invisit=true; s_dayslastvisit_s=First%20Visit; s_cc=true; aam_uuid=16483151297428366474114464420193254715; _ga=GA1.2.2116246096.1555115690; _gid=GA1.2.1970831125.1555115690; s_prevPage=rt%20%7C%20browse%20%7C%20dvd%20%26%20streaming%20movies%20%7C%20browse%20all; bm_monthly_unique=true; bm_daily_unique=true; bm_sample_frequency=100; WRIgnore=true; _cb=C9GvJVCDWX9uB3tEzA; _cb_svref=null; _v__chartbeat3=ODu4VCnEQ6YB53sKH; __gads=ID=5e3e9edf0b246cb4:T=1555115690:S=ALNI_MYA8RB0F-VNaozhWd1QUFTLG2Xy-A; QSI_HistorySession=https%3A%2F%2Fwww.rottentomatoes.com%2Fbrowse%2Fdvd-streaming-all~1555116187086; check=true; mbox=session#0626514dbede49ccbb07eb4f0fadf213#1555118305|PC#0626514dbede49ccbb07eb4f0fadf213.17_64#1618361245; s_sq=%5B%5BB%5D%5D; mps_uuid=f1668886-8c27-477f-9b51-ba5a5049de7a; s_dayslastvisit=1555117663768; bm_last_load_status=NOT_BLOCKING; __CT_Data=gpv=7&ckp=tld&dm=rottentomatoes.com&apv_228_www12=7&cpv_228_www12=7; _chartbeat2=.1555115690545.1555117665000.1.NWVfhCczon8BwEXeyDlOX--CTn_zv.7; _awl=3.1555117670.0.4-909b19fb-e8ef0df67ef7391cae4f86fb95c1cf29-6763652d75732d63656e7472616c31-5cb13661-0; _gali=show-more-btn',
          'referer': 'https://www.rottentomatoes.com/browse/dvd-streaming-all'}
tomato_item_url = list()
for url in tomato_url:
    text = requests.get(url, headers = headers).text
    js = json.loads(text)
    for element in js['results']:
        tomato_item_url.append('https://www.rottentomatoes.com' + element['url'])

In [5]:
## Extract information from movie url -- Rotten Tomatoes
tomato_result = list()
for url in tomato_item_url:
    item = dict()
    text = requests.get(url, headers = headers).text
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = text.replace('\t', '')
    item['URL'] = url
    item['Name'] = re.search('https://www.rottentomatoes.com/m/(.*)', url).group(1).replace('_', ' ')
    cast = re.search('<section id="movie-cast" class="panel panel-rt panel-box">(((?!</section>).)*)</section>', text)
    if cast is not None:
        stars = re.findall('<span title="([^"]*)">', cast.group(1))
        if stars is not None:
            item['Stars'] = ','.join(stars).strip()
    info = re.search('<ul class="content-meta info">(((?!</ul>).)*)</ul>', text)
    if info is not None:
        info = info.group(1)
        year = re.search('<div class="meta-label subtle">In Theaters: </div> *<div class="meta-value"> *<time datetime="[^<>]*">(((?!</time>).)*)</time>', info)
        if year is not None:
            item['Year'] = year.group(1).strip()
        time = re.search('<div class="meta-label subtle">Runtime: </div> *<div class="meta-value"> *<time datetime="[^<>]*">(((?!</time>).)*)</time>', info)
        if time is not None:
            item['Time'] = time.group(1).strip()
        rating = re.search('<div class="meta-label subtle">Rating: </div> *<div class="meta-value">(((?!</div>).)*)</div>', info)
        if rating is not None:
            item['Rating'] = rating.group(1).strip()
        genre = re.findall('<a href="/browse/opening/\?genres=[0-9]*">(((?!</a>).)*)</a>', info)
        if genre is not None:
            item['Genre'] = ','.join([i[0] for i in genre]).replace('&amp;', '&').strip()
        director = re.search('<div class="meta-label subtle">Directed By: </div> *<div class="meta-value"> *<a href="[^"]*">(((?!</a>).)*)</a>', info)
        if director is not None:
            item['Director'] = director.group(1).strip()
        tomato_result.append(item)

In [6]:
## Save Rotten Tomatoes data into csv
with open('../data/tableB_rottentomatoes.csv', 'w') as f:
    keys = tomato_result[0].keys()
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(tomato_result)